In [4]:
import cv2
import numpy as np

In [5]:
def nothing(x):
    pass

In [6]:
image = cv2.imread(r"C:\Users\prana\Downloads\assignment.png")
img = cv2.resize(image,(400,400))
rect = cv2.selectROI('Makeup', img, False, False)
h1, w1 = int(rect[3]), int(rect[2])
roi = img[int(rect[1]):int(rect[1]+rect[3]), int(rect[0]):int(rect[0]+rect[2])]
cv2.destroyAllWindows()

cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Controls', 500, 600)
cv2.namedWindow('Beautified', cv2.WINDOW_NORMAL)
cv2.namedWindow('Original', cv2.WINDOW_NORMAL)

cv2.createTrackbar('LH', 'Controls', 150, 179, nothing)
cv2.createTrackbar('LS', 'Controls', 50, 255, nothing)
cv2.createTrackbar('LV', 'Controls', 120, 255, nothing)
cv2.createTrackbar('UH', 'Controls', 170, 179, nothing)
cv2.createTrackbar('US', 'Controls', 255, 255, nothing)
cv2.createTrackbar('UV', 'Controls', 255, 255, nothing)
cv2.createTrackbar('Blur', 'Controls', 15, 50, nothing)
cv2.createTrackbar('B', 'Controls', 40, 255, nothing)
cv2.createTrackbar('G', 'Controls', 0, 255, nothing)
cv2.createTrackbar('R', 'Controls', 150, 255, nothing)
cv2.createTrackbar('Tint', 'Controls', 60, 100, nothing)
cv2.createTrackbar('SX', 'Controls', w1 // 2, w1 - 1, nothing)
cv2.createTrackbar('SY', 'Controls', h1 // 2, h1 - 1, nothing)
cv2.createTrackbar('SS', 'Controls', 15, min(h1, w1) // 2, nothing)

while True:
    lh = cv2.getTrackbarPos('LH', 'Controls')
    ls = cv2.getTrackbarPos('LS', 'Controls')
    lv = cv2.getTrackbarPos('LV', 'Controls')
    uh = cv2.getTrackbarPos('UH', 'Controls')
    us = cv2.getTrackbarPos('US', 'Controls')
    uv = cv2.getTrackbarPos('UV', 'Controls')
    b = cv2.getTrackbarPos('B', 'Controls')
    g = cv2.getTrackbarPos('G', 'Controls')
    r = cv2.getTrackbarPos('R', 'Controls')
    tint = cv2.getTrackbarPos('Tint', 'Controls') / 100.0
    sx = cv2.getTrackbarPos('SX', 'Controls')
    sy = cv2.getTrackbarPos('SY', 'Controls')
    ss = cv2.getTrackbarPos('SS', 'Controls')
    blur_amt = cv2.getTrackbarPos('Blur', 'Controls')
    blur_amt = max(3, blur_amt | 1)
    roi_blur = cv2.GaussianBlur(roi, (blur_amt, blur_amt), 0)
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    lower = np.array([lh, ls, lv])
    upper = np.array([uh, us, uv])
    mask = cv2.inRange(hsv, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask_clean = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    color = np.full(roi.shape, (b, g, r), dtype=np.uint8)
    lips = cv2.bitwise_and(color, color, mask=mask_clean)
    makeup = cv2.addWeighted(roi_blur, 1, lips, tint, 0)
    out_roi = roi_blur.copy()
    out_roi[mask_clean > 0] = makeup[mask_clean > 0]
    sx = min(max(sx, 0), w1 - 1)
    sy = min(max(sy, 0), h1 - 1)
    ss = min(ss, w1 // 2, h1 // 2)
    cv2.circle(out_roi, (sx, sy), ss, (255, 0, 0), -1)
    out_img = img.copy()
    out_img[int(rect[1]):int(rect[1]+rect[3]), int(rect[0]):int(rect[0]+rect[2])] = out_roi
    cv2.imshow('Beautified', out_img)
    cv2.imshow('Original', img)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.imwrite('beautified_output.jpg', out_img)
cv2.destroyAllWindows()